# 📘 Anleitung: Starten des Notebooks

Dieses Notebook wird von GitHub geladen. Um die Analyse zu starten, folgen Sie bitte diesen Schritten:

1. **Umgebung vorbereiten:** Gehen Sie oben im Menü auf **Laufzeit** (Runtime) → **Alle ausführen** (Run all).
2. **Warnung bestätigen:** Wenn das Fenster *"Warnung: Dieses Notebook wurde nicht von Google erstellt"* erscheint, klicken Sie auf **„Trotzdem ausführen“**.
3. **Berechnung starten:** Sobald das Bedienfeld erscheint, klicken Sie unten auf die grüne Schaltfläche **▶ Ausführen**.

---
*Die Warnung ist eine Google-Sicherheitsmaßnahme für GitHub-Dateien. Die grüne Schaltfläche trainiert das KI-Modell und sendet die Prognosen an die App.*

In [27]:
# @title 🎮 START CONTROL PANEL (Debug Enhanced)
import ipywidgets as widgets
from IPython.display import display, HTML

# UI
style = """<style>.lotto-box{background:#f8f9fa;padding:15px;border-radius:10px;border:1px solid #ddd;}.lbl{font-weight:bold;font-size:16px;margin-bottom:10px;display:block;}</style>"""
display(HTML(style))

lbl = widgets.HTML('<span class="lbl">🎲 LOTTO 6aus49 KI-Generator</span>')
txt_uid = widgets.Text(placeholder="User ID (Auto)", description="🆔 ID:", layout=widgets.Layout(width='200px'))
btn_go = widgets.Button(description="Start Pipeline", button_style='success', icon='rocket', layout=widgets.Layout(width='150px'))
out_log = widgets.Output(layout={'border':'1px solid #ccc','height':'150px','overflow':'auto','margin_top':'10px'})
progress = widgets.IntProgress(value=0, max=100, layout=widgets.Layout(width='100%'))

# Bridge
global_ui = { "btn": btn_go, "txt": txt_uid, "log": out_log, "bar": progress }

def on_click_wrapper(b):
    if 'run_lotto_logic' in globals():
        globals()['run_lotto_logic'](global_ui)
    else:
        with out_log:
            print("⚠️ Bitte Zelle 2 (Code) auch ausführen!")

btn_go.on_click(on_click_wrapper)

display(widgets.VBox([lbl, widgets.HBox([txt_uid, btn_go]), progress, out_log]))

# --- STRONG AUTO DETECT ---
from google.colab import output
js_debug = """
(function(){
  try {
      var url = window.location.href;
      var hash = window.location.hash;
      return {url: url, hash: hash};
  } catch(e) { return {error: e.toString()}; }
})();
"""

try:
    # 1. Try to read
    data = output.eval_js(js_debug)

    # 2. Parse Logic
    hash_str = data.get('hash', '')
    found_id = ""

    # Parse #userId=123
    if 'userId=' in hash_str:
        parts = hash_str.split('userId=')
        if len(parts) > 1:
            found_id = parts[1].split('&')[0]

    if found_id:
        txt_uid.value = str(found_id)
        with out_log: print(f"✅ Auto-Login: User {found_id}")
    else:
        with out_log:
            print("ℹ️ Kein User ID im Link gefunden.")
            # print(f"Debug URL: {data.get('url')}") # Optional for debug
            print("👉 Bitte öffnen Sie Colab über den Link in der App!")

except Exception as e:
    with out_log: print(f"Debug Error: {e}")

SyntaxError: invalid syntax (ipython-input-1391526803.py, line 20)

In [ ]:
# @title ⚙️ CORE SYSTEM (Must actviate once)
import time, requests, json, zipfile, io, os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import Callback

BACKEND = "https://apilotto.euroceiling39.ru"

def run_lotto_logic(ui):
    btn, txt, log_out, bar = ui['btn'], ui['txt'], ui['log'], ui['bar']

    btn.disabled = True
    bar.value = 0
    log_out.clear_output()

    def log(m):
        with log_out: print(f"[{time.strftime('%H:%M:%S')}] {m}")

    uid = txt.value.strip()
    if not uid:
        log("❌ FEHLER: Keine User ID!")
        btn.disabled = False
        return

    try:
        log(f"🚀 Starte für User: {uid}")
        bar.value = 5

        # 1. Daten laden
        log("📥 Lade Archiv...")
        r = requests.get("https://www.lotto-bayern.de/static/gamebroker_2/de/download_files/archiv_lotto.zip", timeout=30)
        with zipfile.ZipFile(io.BytesIO(r.content)) as z: z.extractall("/content/lotto_temp")

        rows=[]
        with open("/content/lotto_temp/lotto.txt", "r", encoding="utf-8", errors="ignore") as f:
            for l in f.readlines()[1:]:
                p = l.split()
                if len(p)>10 and int(p[2])>=2013: rows.append(p[3:10])

        df = pd.DataFrame(rows).apply(pd.to_numeric, errors='coerce').dropna()
        vals = df.values; vals[:,:6] = np.sort(vals[:,:6], axis=1)

        # 2. Training
        log("🧠 Training KI Modell...")
        bar.value = 20

        scaler = StandardScaler().fit(vals)
        scaled = scaler.transform(vals)
        win = 24
        X = np.array([scaled[i:i+win] for i in range(len(scaled)-win)])
        y = np.array([scaled[i+win] for i in range(len(scaled)-win)])

        model = Sequential([
            Bidirectional(LSTM(128, return_sequences=True), input_shape=(win,7)),
            Dropout(0.2), Bidirectional(LSTM(64)), Dense(7)
        ])
        model.compile(optimizer='adam', loss='mse')

        class P(Callback):
            def on_epoch_end(s,e,l): bar.value = 20 + int(70 * (e/60))

        model.fit(X, y, epochs=60, batch_size=64, verbose=0, callbacks=[P()])

        # 3. Prognose
        last = scaled[-win:].reshape(1,win,7)
        pred = scaler.inverse_transform(model.predict(last))[0]
        nums = sorted([min(49,max(1,int(round(x)))) for x in pred[:6]])
        sz = min(9,max(0,int(round(pred[6]))))

        log(f"✅ Prognose: {nums} [SZ: {sz}]")
        bar.value = 95

        # 4. Senden
        log("📡 Sende an App...")
        res = requests.post(f"{BACKEND}/api/colab/update", json={
            "userId": uid, "numbers": nums, "generated_at": int(time.time())
        })

        if res.ok:
            log("🎉 ERFOLG! Daten in App verfügbar.")
            bar.value = 100
        else:
            log(f"⚠️ Server Fehler: {res.status_code}")

    except Exception as e:
        log(f"Error: {e}")

    btn.disabled = False